In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Load

In [2]:
df_100k = pd.read_csv("../raw_data/Meteo/donnees-synop-essentielles-omm.csv", sep=";", nrows=100_000)

In [22]:
df_100k.head()

,ID OMM station,Date,Pression au niveau mer,Variation de pression en 3 heures,Type de tendance barométrique,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Point de rosée,Humidité,...,communes (name),communes (code),EPCI (name),EPCI (code),department (name),department (code),region (name),region (code),mois_de_l_annee,Year
0,81405,2013-04-03T17:00:00+02:00,101460.0,NaN,NaN,40.0,7.2,302.75,293.85,59.0,...,Matoury,97307,CA du Centre Littoral,249730045.0,Guyane,973,Guyane,3.0,4,2013
1,7027,2013-04-03T20:00:00+02:00,101180.0,-40.0,7,20.0,5.7,277.15,272.75,73.0,...,Carpiquet,14137,CU Caen la Mer,200065597.0,Calvados,14,Normandie,28.0,4,2013
2,7181,2013-04-03T20:00:00+02:00,100890.0,20.0,3,70.0,7.7,278.25,271.15,60.0,...,Thuilley-aux-Groseilles,54523,CC du Pays de Colombey et du Sud Toulois,245400510.0,Meurthe-et-Moselle,54,Grand Est,44.0,4,2013
3,7747,2013-04-03T20:00:00+02:00,100350.0,-100.0,6,170.0,1.5,286.75,281.95,73.0,...,Perpignan,66136,CU Perpignan Méditerranée Métropole,200027183.0,Pyrénées-Orientales,66,Occitanie,76.0,4,2013
4,7207,2013-04-03T23:00:00+02:00,100950.0,100.0,1,30.0,10.3,276.95,271.75,69.0,...,Bangor,56009,CC de Belle Ile en Mer,245600465.0,Morbihan,56,Bretagne,53.0,4,2013


# Clean

In [25]:
df_100k['Year'] = df_100k['Date'].str[:4]
df_100k['Month'] = df_100k['Date'].str[5:7]
df_100k['Day'] = df_100k['Date'].str[8:10]
df_100k['Hour'] = df_100k['Date'].str[11:13]

In [39]:
df_100k['DateClean'] = pd.to_datetime(df_100k['Year']+df_100k['Month']+df_100k['Day'])

In [40]:
df_7027 = df_100k[df_100k['ID OMM station']==7027]

# Dpts

In [107]:
df_dpts = pd.read_excel("../raw_data/Meteo/Classement_Departement.xlsx")

In [108]:
df_dpts = df_dpts.dropna()

In [109]:
df_dpts

,Departement,Production,Part Cumul,ID OMM station
0,80 - Somme,1682.43880,0.111671,7005.0
1,51 - Marne,1416.11300,0.205665,7072.0
2,59 - Nord,1034.29740,0.274316,7015.0
3,76 - Seine-Maritime,1011.94720,0.341483,7037.0
4,10 - Aube,847.57400,0.397741,7168.0
5,86 - Vienne,822.28150,0.452319,7335.0
6,37 - Indre-et-Loire,672.49306,0.496956,7240.0
7,35 - Ille-et-Vilaine,663.43710,0.540991,7130.0
8,21 - Côte-d'Or,648.13320,0.584010,7280.0
9,22 - Côtes-d'Armor,646.42780,0.626917,7117.0


In [110]:
df_dpts['ID OMM station'] = df_dpts['ID OMM station'].astype("int")

In [111]:
df_dpts

,Departement,Production,Part Cumul,ID OMM station
0,80 - Somme,1682.43880,0.111671,7005
1,51 - Marne,1416.11300,0.205665,7072
2,59 - Nord,1034.29740,0.274316,7015
3,76 - Seine-Maritime,1011.94720,0.341483,7037
4,10 - Aube,847.57400,0.397741,7168
5,86 - Vienne,822.28150,0.452319,7335
6,37 - Indre-et-Loire,672.49306,0.496956,7240
7,35 - Ille-et-Vilaine,663.43710,0.540991,7130
8,21 - Côte-d'Or,648.13320,0.584010,7280
9,22 - Côtes-d'Armor,646.42780,0.626917,7117


In [115]:
df_100k_clean = df_100k.merge(df_dpts, on="ID OMM station", how="inner")

# Explore

In [121]:
df_100k_clean['Etat du sol'].count()

12495

In [123]:
df_100k_clean.groupby(["DateClean", "ID OMM station"]).sum()['Etat du sol']

DateClean   ID OMM station
2010-01-01  7020              0.0
            7027              0.0
            7117              0.0
            7130              0.0
            7139              0.0
                             ... 
2018-05-31  7168              0.0
            7181              0.0
            7255              0.0
            7335              0.0
            7460              0.0
Name: Etat du sol, Length: 29263, dtype: float64

In [126]:
df_100k_clean.groupby("ID OMM station").count()["Etat du sol"].sort_values(ascending=False)

ID OMM station
7015    820
7460    804
7240    779
7222    774
7110    767
7280    766
7190    738
7335    730
7027    727
7139    723
7630    709
7168    705
7130    637
7005    615
7072    601
7314    551
7255    336
7117    307
7037    278
7207     67
7181     43
7020     18
Name: Etat du sol, dtype: int64

In [124]:
29263/12495

2.3419767907162865

In [54]:
df_100k[df_100k['ID OMM station']==7747].sort_values(by="DateClean")

,ID OMM station,Date,Pression au niveau mer,Variation de pression en 3 heures,Type de tendance barométrique,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Point de rosée,Humidité,...,department (name),department (code),region (name),region (code),mois_de_l_annee,Year,Month,Day,Hour,DateClean
49579,7747,2010-01-01T19:00:00+01:00,100570.0,310.0,"En hausse, puis stationnaire, ou en hausse, pu...",310.0,8.2,283.85,275.15,55.0,...,Pyrénées-Orientales,66,Occitanie,76.0,1,2010,01,01,19,2010-01-01
49560,7747,2010-01-01T01:00:00+01:00,99140.0,-40.0,5,310.0,5.7,283.65,279.55,76.0,...,Pyrénées-Orientales,66,Occitanie,76.0,1,2010,01,01,01,2010-01-01
21750,7747,2010-01-01T10:00:00+01:00,99580.0,280.0,En hausse,300.0,11.8,285.15,271.65,39.0,...,Pyrénées-Orientales,66,Occitanie,76.0,1,2010,01,01,10,2010-01-01
49621,7747,2010-01-03T16:00:00+01:00,101750.0,-120.0,5,20.0,3.6,279.55,274.65,71.0,...,Pyrénées-Orientales,66,Occitanie,76.0,1,2010,01,03,16,2010-01-03
49544,7747,2010-01-04T10:00:00+01:00,101360.0,50.0,"En hausse, puis stationnaire, ou en hausse, pu...",330.0,1.0,277.85,274.85,81.0,...,Pyrénées-Orientales,66,Occitanie,76.0,1,2010,01,04,10,2010-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34330,7747,2018-05-25T23:00:00+02:00,101510.0,90.0,3,80.0,2.2,291.95,286.15,69.0,...,Pyrénées-Orientales,66,Occitanie,76.0,5,2018,05,25,23,2018-05-25
34246,7747,2018-05-28T11:00:00+02:00,101330.0,60.0,0,330.0,3.2,291.95,289.15,84.0,...,Pyrénées-Orientales,66,Occitanie,76.0,5,2018,05,28,11,2018-05-28
81077,7747,2018-05-28T08:00:00+02:00,101270.0,-20.0,"Stationnaire ou en hausse, puis en baisse, ou ...",350.0,3.2,292.55,288.45,77.0,...,Pyrénées-Orientales,66,Occitanie,76.0,5,2018,05,28,08,2018-05-28
81121,7747,2018-05-31T17:00:00+02:00,101690.0,-30.0,6,120.0,4.7,296.55,288.05,59.0,...,Pyrénées-Orientales,66,Occitanie,76.0,5,2018,05,31,17,2018-05-31


In [87]:
df_100k['DateClean'].nunique()

3042

In [91]:
ets.sort_index()

0.0     15634
1.0      4294
2.0      3377
3.0         3
4.0       314
5.0       142
6.0         1
9.0         1
10.0        2
11.0      128
12.0      200
13.0      347
14.0       60
15.0       34
16.0       59
17.0       98
18.0       66
19.0        7
Name: Etat du sol, dtype: int64

In [86]:
df_100k.groupby("ID OMM station").count()["Etat du sol"].sort_values(ascending=False)

ID OMM station
7299     905
7481     859
7149     824
7015     820
7460     804
        ... 
61970      0
61998      0
61997      0
61996      0
89642      0
Name: Etat du sol, Length: 62, dtype: int64

In [88]:
3042/900

3.38

In [83]:
24767/78373

0.31601444374976073

In [125]:
ets = df_100k['Etat du sol'].value_counts()

In [76]:
ets.sort_index().sum()

24767

In [4]:
df_100k.columns

Index(['ID OMM station', 'Date', 'Pression au niveau mer',
       'Variation de pression en 3 heures', 'Type de tendance barométrique',
       'Direction du vent moyen 10 mn', 'Vitesse du vent moyen 10 mn',
       'Température', 'Point de rosée', 'Humidité', 'Visibilité horizontale',
       'Temps présent', 'Temps passé 1', 'Temps passé 2', 'Nebulosité totale',
       'Nébulosité  des nuages de l' étage inférieur',
       'Hauteur de la base des nuages de l'étage inférieur',
       'Type des nuages de l'étage inférieur',
       'Type des nuages de l'étage moyen',
       'Type des nuages de l'étage supérieur', 'Pression station',
       'Niveau barométrique', 'Géopotentiel',
       'Variation de pression en 24 heures',
       'Température minimale sur 12 heures',
       'Température minimale sur 24 heures',
       'Température maximale sur 12 heures',
       'Température maximale sur 24 heures',
       'Température minimale du sol sur 12 heures',
       'Méthode de mesure Température du